# Домашнее задание к лекции "Продвинутый pandas 2"

## Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd

ratings = pd.read_csv(r'C:\Users\RyzhikAV\PYDA-7\ml-latest-small\ratings.csv')
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit = 's')

In [37]:
ratings_metric = ratings.groupby(ratings.userId).agg({'date': ['min', 'max'], 'rating': 'count'}).reset_index()

In [63]:
ratings_metric_100more = ratings_metric[ratings_metric ['rating'] ['count'] > 100].reset_index()

In [105]:
ratings_metric_100more['delta_time'] = (ratings_metric_100more ['date'] ['max'] 
                                        - ratings_metric_100more ['date'] ['min']).astype('timedelta64[m]')

In [117]:
lifetime_days = round(ratings_metric_100more.delta_time.mean()/60/24)

In [118]:
lifetime_days

464

## Задание 2

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки

auto - автомобильные перевозки

air - воздушные перевозки

client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [160]:
revenue_all_types = (
    rzd
     .merge(air, how='outer', on='client_id')
     .merge (auto, how='outer', on='client_id')
    )

In [161]:
revenue_all_types

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,57483.0
3,114,5774.0,NaN,83.0
4,115,981.0,81.0,912.0
5,116,NaN,4.0,4834.0
6,117,NaN,13.0,98.0
7,118,NaN,173.0,NaN


In [162]:
revenue_all_types.merge (client_base, how='outer', on='client_id')

,client_id,rzd_revenue,air_revenue,auto_revenue,address
0,111,1093.0,NaN,NaN,Комсомольская 4
1,112,2810.0,NaN,NaN,Энтузиастов 8а
2,113,10283.0,NaN,57483.0,Левобережная 1а
3,114,5774.0,NaN,83.0,Мира 14
4,115,981.0,81.0,912.0,ЗЖБИиДК 1
5,116,NaN,4.0,4834.0,Строителей 18
6,117,NaN,13.0,98.0,Панфиловская 33
7,118,NaN,173.0,NaN,Мастеркова 4


## Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Я бы посчитал моду по координатам;
Посчитал среднее значение координат для всего спсика юзеров;
Посчитал бы количество повторений координат для каждого юзера;
Если все координаты уникальны (повторений не было) , то брал бы координаты максимально приближенные к среднему
Иначе - моду;
Далее использовал бы долготу в одном столбце, широту - в другом
Таким образом для каждого user_id был один вариант широты и долготы